# Imports

In [79]:
import numpy as np
import pandas as pd

from functools import reduce


import plotly.express as px


import warnings
warnings.filterwarnings('ignore')

In [80]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


In [81]:
df_dictionary = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

# Country Breakdown

In [85]:
use_df = df_dictionary['2022'][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

In [86]:

use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

In [92]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

In [93]:
use_df['clim_rel'] = use_df.apply(determine_value, axis=1)


In [98]:
use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']

In [104]:
grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]

In [107]:
grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']

In [108]:
grouped_df

,Amount,clim_rel_amount,clim_rel_percent
Recipient Name,,,
Afghanistan,433.085734,12.035070,0.027789
"Africa, regional",948.405609,187.550717,0.197754
Albania,91.201742,23.199558,0.254376
Algeria,22.634255,7.174906,0.316993
"America, regional",362.063541,79.349972,0.219160
...,...,...,...
West Bank and Gaza Strip,260.859632,17.386640,0.066651
"Western Africa, regional",149.138587,25.979593,0.174198
Yemen,489.529717,4.985232,0.010184


In [113]:
processed_grouped_dfs = []

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

for key in df_dictionary.keys():
        
    use_df = df_dictionary[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']].reset_index()
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_processed_df = grouped_df.rename(columns={'clim_rel_percent': key})

    processed_grouped_dfs.append(grouped_processed_df[['Recipient Name',key]])

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


# df_long.to_csv('../upload_data/country_specific_df.csv')



In [118]:
df_long.to_csv('../upload_data/newmethod_country_specific_df.csv')


In [82]:
processed_grouped_dfs = []

for key in df_dictionary.keys():
        
    use_df = df_dictionary[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)

    use_df['Climate Relevance'] = use_df['Climate Adaptation'] + use_df['Climate Mitigation']

    use_df.loc[use_df['Climate Relevance'] > 1, 'Climate Relevance'] = 1

    first_group_df = use_df.groupby(['Recipient Name', 'Climate Relevance']).sum()[['Amount']].reset_index()

    df_grouped = first_group_df.groupby('Recipient Name').apply(
        lambda x: (x[x['Climate Relevance'] == 1]['Amount'].sum()\
                   / x['Amount'].sum()) * 100).reset_index(name='Climate Relevance %')

    grouped_processed_df = df_grouped.rename(columns={'Climate Relevance %': key})

    processed_grouped_dfs.append(grouped_processed_df)

    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)

df_long = df_merged.melt(id_vars=["Recipient Name"], 
                  var_name="Year", 
                  value_name="Value")


df_long.to_csv('../upload_data/country_specific_df.csv')



In [72]:
df_long

,Recipient Name,Year,Value
0,Afghanistan,2013,4.971200
1,"Africa, regional",2013,38.460540
2,Albania,2013,43.069359
3,Algeria,2013,28.187026
4,"America, regional",2013,53.594152
...,...,...,...
1645,"Melanesia, regional",2022,100.000000
1646,"Middle Africa, regional",2022,44.067499
1647,"Southern Africa, regional",2022,90.432986
1648,"Western Africa, regional",2022,24.897512


In [73]:
df_long = pd.read_csv('../upload_data/country_specific_df.csv')[['Recipient Name', 'Year', 'Value']]

In [74]:
df_long

,Recipient Name,Year,Value
0,Afghanistan,2013,4.971200
1,"Africa, regional",2013,38.460540
2,Albania,2013,43.069359
3,Algeria,2013,28.187026
4,"America, regional",2013,53.594152
...,...,...,...
1645,"Melanesia, regional",2022,100.000000
1646,"Middle Africa, regional",2022,44.067499
1647,"Southern Africa, regional",2022,90.432986
1648,"Western Africa, regional",2022,24.897512


In [53]:
countries = df_long['Recipient Name'].unique()
# countries

In [116]:
selected_df = df_long[df_long['Recipient Name'].isin(['India', 'Brazil', 'Ukraine', 'Namibia', 'South Africa'])]

In [117]:
fig = px.line(selected_df, 
              x='Year', 
              y='Value', 
              color='Recipient Name',
              title='Value by Year for Selected Country',
              labels={'Value': 'Climate Relevant Financing (%)', 'Year': 'Year'},
              markers=True)

fig.show()

# Globe Graph

In [4]:
def df_compiler(dictionary_of_df):
    
    processed_grouped_dfs = []
    
    for key in dictionary_of_df.keys():
        
        use_df = dictionary_of_df[key][['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
        
        use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
        use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
        
        use_df['Climate Relevance'] = use_df['Climate Adaptation'] + use_df['Climate Mitigation']
        
        use_df.loc[use_df['Climate Relevance'] > 2, 'Climate Relevance'] = 2
        
        use_df['Amount'] = use_df['Amount'] * 1_000_000
        
        grouped_processed_df = use_df.groupby(['Climate Relevance']).sum()[['Amount']]
        
        grouped_processed_df = grouped_processed_df.rename(columns={'Amount': key})
        
        processed_grouped_dfs.append(grouped_processed_df)
        
    merged_df = pd.concat(processed_grouped_dfs, axis=1)
    
    sum_row = merged_df.sum()
    sum_df = pd.DataFrame([sum_row], index=['Total Financing'])
    
    merged_df = merged_df.append(sum_df)

    merged_df.index = ['No Climate Relevance','Partial Climate Relevance',
                       'High Climate Relevance', 'Total Financing']

    return merged_df


merged_df = df_compiler(df_dictionary)

merged_df.to_csv('../upload_data/global_df.csv')

merged_df

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
No Climate Relevance,8.626761e+09,1.094957e+10,1.467575e+10,2.011983e+10,2.061496e+10,1.826632e+10,1.709581e+10,2.059103e+10,2.112636e+10,2.574590e+10
Partial Climate Relevance,1.351645e+09,1.079722e+09,1.810979e+09,2.011495e+09,1.491296e+09,2.153488e+09,1.766962e+09,2.535233e+09,2.168643e+09,2.867661e+09
High Climate Relevance,1.911409e+09,2.636901e+09,2.598213e+09,3.636717e+09,3.863953e+09,3.656074e+09,4.727203e+09,4.549515e+09,4.367615e+09,6.337111e+09
Total Financing,1.188982e+10,1.466620e+10,1.908495e+10,2.576805e+10,2.597021e+10,2.407588e+10,2.358997e+10,2.767578e+10,2.766262e+10,3.495068e+10


In [7]:
globe_df = pd.read_csv('../upload_data/global_df.csv')
globe_df

,Unnamed: 0,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,No Climate Relevance,8.626761e+09,1.094957e+10,1.467575e+10,2.011983e+10,2.061496e+10,1.826632e+10,1.709581e+10,2.059103e+10,2.112636e+10,2.574590e+10
1,Partial Climate Relevance,1.351645e+09,1.079722e+09,1.810979e+09,2.011495e+09,1.491296e+09,2.153488e+09,1.766962e+09,2.535233e+09,2.168643e+09,2.867661e+09
2,High Climate Relevance,1.911409e+09,2.636901e+09,2.598213e+09,3.636717e+09,3.863953e+09,3.656074e+09,4.727203e+09,4.549515e+09,4.367615e+09,6.337111e+09
3,Total Financing,1.188982e+10,1.466620e+10,1.908495e+10,2.576805e+10,2.597021e+10,2.407588e+10,2.358997e+10,2.767578e+10,2.766262e+10,3.495068e+10


In [9]:

df_long = merged_df.stack().reset_index()
df_long.columns = ['Climate Relevance', 'Year', 'Financing']
df_long = df_long[df_long['Climate Relevance'] != 'Total Financing']



fig = px.bar(
    df_long,
    x='Year',
    y='Financing',
    color='Climate Relevance',  # This determines the stack segments
    hover_data=['Climate Relevance', 'Financing'],  # Customize hover info
    title='Global BMZ Financing by Climate Relevance'
)

# Adjust layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Financing',
    barmode='stack'
)

# Show the figure
fig.show()

In [ ]:
df_long = df.stack().reset_index()
df_long.columns = ['Climate Relevance', 'Year', 'Financing']

# Filter out the 'Total Financing' row to avoid double counting in the stacked bar
df_long = df_long[df_long['Climate Relevance'] != 'Total Financing']

# Create the stacked bar chart
fig = px.bar(
    df_long,
    x='Year',
    y='Financing',
    color='Climate Relevance',  # This determines the stack segments
    hover_data=['Climate Relevance', 'Financing'],  # Customize hover info
    title='Total Financing by Climate Relevance'
)

# Adjust layout
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Financing',
    barmode='stack'
)

# Show the figure
fig.show()

In [ ]:


testing_2022 = df_2022[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]

In [ ]:
testing_2022['Climate Adaptation'] = testing_2022['Climate Adaptation'].fillna(0)
testing_2022['Climate Mitigation'] = testing_2022['Climate Mitigation'].fillna(0)

In [ ]:
testing_2022['Climate Relevance'] = testing_2022['Climate Adaptation'] + testing_2022['Climate Mitigation']

In [ ]:
testing_2022.loc[testing_2022['Climate Relevance'] > 2, 'Climate Relevance'] = 2


In [ ]:
testing_2022.groupby(['Climate Relevance']).sum()[['Amount']].rename(columns={'Amount': '2022'})

In [ ]:
grouped_test = testing_2022.groupby(['Recipient Name', 'Climate Relevance']).sum()[['Amount']]